In [1]:
import numpy as np
import datetime
import os
import copy
# Generate WindowFrame Function
from mymodule import window
import joblib
import pandas as pd

# Parametor about Sensor
SensorNum=1 # Sensor number
#SensorAxis='AccX' # Axis

# Define Data Name
DATE= str( datetime.date.today() )+'/'

# Parametor about " Data Path"
StorePath = "/media/takeyama/Transfer/02_ActivityResearch/"

ParticipantsNum="Participants07/"
Version = "03_Third/"
Time = "01_First/"
CommonName = 'Participants07-Third-01'
LabelName = CommonName+'.csv'

TITLE = ParticipantsNum+Version

DataPath = StorePath+ParticipantsNum+Version+"RawData/"+Time

if not os.path.exists(StorePath+TITLE+'LabelData/'+Time): 
    os.makedirs(StorePath+TITLE+'LabelData/'+Time)
LabelDataPath=StorePath+TITLE+'LabelData/'+Time

if not os.path.exists(StorePath+TITLE+'dictionary/'+Time): 
    os.makedirs(StorePath+TITLE+'dictionary/'+Time)
DictionaryDataPath=StorePath+TITLE+'dictionary/'+Time

if not os.path.exists(StorePath+TITLE+'feature/'+Time): 
    os.makedirs(StorePath+TITLE+'feature/'+Time)
FeaturePath=StorePath+TITLE+'feature/'+Time

if not os.path.exists(StorePath+TITLE+'window/'+Time): 
    os.makedirs(StorePath+TITLE+'window/'+Time)
WindowDataPath=StorePath+TITLE+'window/'+Time

if not os.path.exists(StorePath+TITLE+DATE+'studyOutput/'+Time): 
    os.makedirs(StorePath+TITLE+DATE+'studyOutput/'+Time)
StudyOutputPath=StorePath+TITLE+DATE+'studyOutput/'+Time

if not os.path.exists(StorePath+TITLE+DATE+'parametor/'+Time): 
    os.makedirs(StorePath+TITLE+DATE+'parametor/'+Time)
ParametorPath=StorePath+TITLE+DATE+'parametor/'+Time

if not os.path.exists(StorePath+TITLE+DATE+'result/'+Time): 
    os.makedirs(StorePath+TITLE+DATE+'result/'+Time)
ResultPath=StorePath+TITLE+DATE+'result/'+Time

if not os.path.exists(StorePath+TITLE+DATE+'graph/'+Time): 
    os.makedirs(StorePath+TITLE+DATE+'graph/'+Time)
GraphPath=StorePath+TITLE+DATE+'graph/'+Time

# 1 Read SensorData

In [2]:
# センサデータのcsvファイル名をリストに集約する関数
def SensorDataFileNameList(path):
    l = os.listdir(DataPath)
    FilePath =[]
    SensorName = []
    
    for n in range(len(l)):
        if (l[n][:4].find('mem-') != -1):
            FilePath.append(l[n])
            SensorName.append(l[n][4:15])
    
    return FilePath,SensorName

In [3]:
# FFT library
from scipy import fftpack
from scipy import signal

# Graph Plot
import matplotlib.pyplot as plt
from matplotlib import gridspec

# Generate WindowFrame Function
from mymodule import window

# timer
import time

# temp file 
import tempfile

%matplotlib inline

In [4]:
DictName = os.listdir(DictionaryDataPath)
print DictName
Sensor1 = np.load(DictionaryDataPath+DictName[0])['data'][()]
Sensor2 = np.load(DictionaryDataPath+DictName[1])['data'][()]
Sensor3 = np.load(DictionaryDataPath+DictName[2])['data'][()]
Sensor4 = np.load(DictionaryDataPath+DictName[3])['data'][()]
Sensor5 = np.load(DictionaryDataPath+DictName[4])['data'][()]
Sensor6 = np.load(DictionaryDataPath+DictName[5])['data'][()]

['01_LeftHand.npz', '02_RightHand.npz', '03_LeftLeg.npz', '04_Rightleg.npz', '05_West.npz', '06_Chest.npz', '07_LabelData.npz']


In [5]:
def GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH):
    """
    data　ウィンドウフレームに変換するデータ
    registName　ウィンドウフレームに登録するデータの名前
    windowWidth　ウィンドウ幅
    slidingWidth　スライド幅
    PATH ウィドウフレームを保存するディレクトリ
    
    
    About Function:
        与えられたdataからウィンドウ幅windowWidth,スライド幅slidingWidthにしたがって
        registNameのウィドウフレームを返す．
        また，与えれたPATH内に同様なパラメータ( WindowWidth, slidingWidth)かつ同様な
        windowNameのものがある場合，そのデータを返す．
        この関数が登録，ウィンドウフレームに変換できるデータは１つとする
    """
    storedName = windowName+'_Win='+str(wWidth).zfill(4)+'_Sld='+str(sWidth).zfill(4)+'.npz'
    l = os.listdir(PATH)
    
    if storedName in l:
        print "this data had finished making"
        return np.load(PATH+storedName)['data'][()]
    
    w=window()
    w.SetData(windowName,dataRaw)    
    wind=w.Compile(wWidth,sWidth)
    windoW=wind.reshape((len(wind),np.prod(wind.shape[1:])))
    
    np.savez(PATH+storedName,data=wind)

    return wind

# Define Extraction Function

# Frequency Domain

In [6]:
def Vector_FFT_Image(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    
    fftwind = np.fft.fft(wind)  # calcurate fast fourier Transfromation
    fftwind = fftwind[:,1:fftwind.shape[1]/2-1] # Orthogonal and Colossus removed
    
        
    # windowFrame(raw wave) 2d -> windowFrame(power) 1d
    Image=np.array([ np.imag(fftwind[i,l])
            for i in range( fftwind.shape[0] )
            for l in range( fftwind.shape[1] ) ])
    Image = Image.reshape(fftwind.shape)    
    # calcurate Squared Energy
    print '===========finished Making Vector of FFT Image==========='
    return Image

In [7]:
def Vector_FFT_Real(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    
    fftwind = np.fft.fft(wind)  # calcurate fast fourier Transfromation
    fftwind = fftwind[:,1:fftwind.shape[1]/2-1] # Orthogonal and Colossus removed
    
        
    # windowFrame(raw wave) 2d -> windowFrame(power) 1d
    Real=np.array([ np.real(fftwind[i,l])
            for i in range( fftwind.shape[0] )
            for l in range( fftwind.shape[1] ) ])
    Real = Real.reshape(fftwind.shape)    
    # calcurate Squared Energy
    print '===========finished Making Vector of FFT Real==========='
    return Real

In [8]:
def Vector_Energy(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    
    fftwind = np.fft.fft(wind)  # calcurate fast fourier Transfromation
    fftwind = fftwind[:,1:fftwind.shape[1]/2-1] # Orthogonal and Colossus removed
      
    # windowFrame(raw wave) 2d -> windowFrame(power) 1d
    SqrF=np.array([
            np.real(fftwind[i,l])**2+np.imag(fftwind[i,l])**2
            for i in range( fftwind.shape[0] )
            for l in range( fftwind.shape[1] )
        ])
    SqrF = SqrF.reshape(fftwind.shape) 
    
    # Sum each window frame
    SumSrqF = np.array([SqrF[i,:].sum() for i in range( SqrF.shape[0])])
    
    # normarise
    NormSumSqrF = np.array([SumSrqF[i]/(fftwind.shape[1]/2-1)
                           for i in range(SumSrqF.shape[0])])

    # root
    RootNormSumSqrF = NormSumSqrF**0.5

    print '===========finished Making Vector of Energy==========='
    return RootNormSumSqrF

In [9]:
def Vector_Frequency(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    print wind.shape
    
    fftwind = np.fft.fft(wind)  # calcurate fast fourier Transfromation
    print fftwind.shape

    fftwind = fftwind[:,1:fftwind.shape[1]/2-1] # Orthogonal and Colossus removed
    print fftwind.shape
    freq = np.array([np.arctan2(np.imag(fftwind[i,l]),np.real(fftwind[i,l]))
                     for i in range( fftwind.shape[0] ) 
                     for l in range( fftwind.shape[1] ) ])
    freq = freq.reshape(fftwind.shape)
    print '===========finished Making Vector of Frequency==========='
    return freq

In [10]:
def Vector_Entropy(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    
    ###########################
    ###    calcurate FFT    ###
    ###########################
    fftwind = np.fft.fft(wind)  # calcurate fast fourier Transfromation
    fftwind = fftwind[:,1:fftwind.shape[1]/2-1] # Orthogonal and Colossus removed
    
    # calcurate eq
    f = lambda x: np.real(x)**2+np.imag(x)**2
        
    # windowFrame(raw wave) 2d -> windowFrame(power) 1d
    SqrF=np.array([
            np.real(fftwind[i,l])**2+np.imag(fftwind[i,l])**2
            for i in range( fftwind.shape[0] )
            for l in range( fftwind.shape[1] )
        ])
    SqrF = SqrF.reshape(fftwind.shape)
    
    # calcurate Squared Energy
    SqrF=SqrF/SqrF[0,:].size
    print SqrF.shape
    
    ##############################
    ###    calcurate Entropy   ###
    ##############################
    P = np.array([SqrF[i]/SqrF[i,:].sum() for i in range(SqrF.shape[0])])
    print P.shape
    P = P.reshape(SqrF.shape)
    
    P_logP = np.array([ P[i,l]*np.log(P[i,l]) \
                      for i in range(P.shape[0]) for l in range(P.shape[1])])
    print P_logP.shape
    P_logP = P_logP.reshape(P.shape)
    
    Entropy = np.array([
            -1*np.sum( P_logP[i,:] ) for i in range(P.shape[0])])
    print '===========finished Making Vector of Entropy==========='
    return Entropy

In [11]:
def Vector_KLD(dataName, dataX, dataY, wWidth, sWidth, PATH):
    """
        Calculates Kullback–Leibler divergence
        input dataX dataY -> 1d Vector
    """
    func_kld =lambda p,q: np.sum(p * np.log(p / q))
    
    KLD = np.array([ func_kld(dataX[i,:],dataY[i,:])\
                    for i in range(dataX.shape[0])])
    print '===========finished Making Vector of KLD==========='
    return KLD

In [12]:
def Vector_JSD(dataName, dataX, dataNameY, dataY, wWidth, sWidth, PATH):
    
    func_kld =lambda p,q: np.sum(p * np.log(p / q))
    func_jsd =lambda p,q: 0.5 * func_kld(p, (p+q)/2) + 0.5 * func_kld(q, (p+q)/2)

    JSD = np.array([ func_jsd(dataX[i,:],dataY[i,:])\
                    for i in range(dataX.shape[0])])
    print '===========finished Making Vector of JSD==========='
    return JSD

# Methematical/Statistical Functions

In [13]:
def Vector_Mean(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    Output = np.array( [wind[i,:].mean() for i in range(wind.shape[0]) ] )
    print '===========finished Making Vector of Mean==========='
    return Output

In [14]:
def Vector_Median(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    Output = np.array( [np.median(wind[i,:]) for i in range(wind.shape[0]) ] )
    print '===========finished Making Vector of Median==========='
    return Output

In [15]:
def Vector_Var(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    Output = np.array( [wind[i,:].var() for i in range(wind.shape[0]) ] )
    print '===========finished Making Vector of Variance==========='
    return Output

In [16]:
def Vector_StdDev(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)  
    Output = np.array( [wind[i,:].var()**0.5 for i in range(wind.shape[0]) ] )
    print '===========finished Making Vector of standard Deviation==========='
    return Output

In [17]:
def Vector_Max(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    Output = np.array( [wind[i,:].max() for i in range(wind.shape[0]) ] )
    print '===========finished Making Vector of Max==========='
    return Output

In [18]:
def Vector_Min(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    Output = np.array( [wind[i,:].min() for i in range(wind.shape[0]) ] )
    print '===========finished Making Vector of Max==========='
    return Output

In [19]:
def Vector_Range(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    Output = np.array( [(wind[i,:].max()-wind[i,:].min())\
                        for i in range(wind.shape[0]) ] )
    print '===========finished Making Vector of Range==========='
    return Output

In [20]:
def Vector_RMS(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    Output = np.array( [( (np.sum( wind[i,:]**2 )/wWidth))**0.5
                        for i in range(wind.shape[0]) ] )
    print '===========finished Making Vector of RMS==========='
    return Output

In [21]:
def Vector_Corr(windowNameX, dataRawX, windowNameY, dataRawY, wWidth, sWidth, PATH):
    # get windowFrame
    windX=GetWindowFrame(windowNameX, dataRawX, wWidth, sWidth, PATH)
    windY=GetWindowFrame(windowNameY, dataRawY, wWidth, sWidth, PATH)
    Output = np.array( [( np.corrcoef(windX[i,:],windY[i,:]) )\
                    for i in range(windX.shape[0]) ] )
    print '===========finished Making Vector of Correlation==========='
    return Output[:,0,1]

In [22]:
def Vector_SMA(windowName, dataRawX, dataRawY, dataRawZ, wWidth, sWidth, PATH):
    # get windowFrame
    windX=GetWindowFrame(windowName, dataRawX, wWidth, sWidth, PATH)
    windY=GetWindowFrame(windowName, dataRawY, wWidth, sWidth, PATH)
    windZ=GetWindowFrame(windowName, dataRawZ, wWidth, sWidth, PATH)
    Output = np.array( [(np.sum((windX[i,:]**2+windY[i,:]**2+windZ[i,:]**2)**0.5))\
                for i in range(windX.shape[0]) ] )
    print '===========finished Making Vector of Signal Magnitude Area==========='
    return Output  

# Extraction

In [23]:
def Extraction(Sensor,FileName,wNum,sNum):
        joblib.dump(Vector_Mean(FileName, Sensor[axis], wNum,sNum, WindowDataPath),
        FeaturePath+FileName+'_Mean.dump')
        
        joblib.dump(Vector_Median(FileName, Sensor[axis], wNum,sNum, WindowDataPath),
                    FeaturePath+FileName+'_Median.dump')
        
        joblib.dump(Vector_Var(FileName, Sensor[axis], wNum,sNum, WindowDataPath),
                    FeaturePath+FileName+'_Var.dump')
        
        joblib.dump(Vector_StdDev(FileName, Sensor[axis], wNum,sNum, WindowDataPath),
                    FeaturePath+FileName+'_StdDev.dump')
        
        joblib.dump(Vector_Max(FileName, Sensor[axis], wNum,sNum, WindowDataPath),
                    FeaturePath+FileName+'_Max.dump' )
        
        joblib.dump(Vector_Min(FileName, Sensor[axis], wNum,sNum, WindowDataPath),
                    FeaturePath+FileName+'_Min.dump')
        
        joblib.dump(Vector_Range(FileName, Sensor[axis], wNum,sNum, WindowDataPath),
                    FeaturePath+FileName+'_Range.dump')
        
        joblib.dump(Vector_RMS(FileName, Sensor[axis], windowNum, slidingNum, WindowDataPath),
                    FeaturePath+FileName+'_RMS.dump' )
        
        joblib.dump(Vector_FFT_Real(FileName, Sensor[axis], wNum,sNum, WindowDataPath),
                    FeaturePath+FileName+'_Real.dump' )

        joblib.dump(Vector_FFT_Image(FileName, Sensor[axis], wNum,sNum, WindowDataPath),
                    FeaturePath+FileName+'_Image.dump' )
       
        joblib.dump(Vector_Energy(FileName, Sensor[axis], wNum,sNum, WindowDataPath),
                    FeaturePath+FileName+'_Energy.dump' )

        joblib.dump(Vector_Entropy(FileName, Sensor[axis], wNum,sNum, WindowDataPath),
                    FeaturePath+FileName+'_Entropy.dump' )

        joblib.dump(Vector_Frequency(FileName, Sensor[axis], wNum,sNum, WindowDataPath),
                    FeaturePath+FileName+'_Frequency.dump' )
        

In [24]:
# 表示用
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook

In [25]:
%%time
windowNum = 256
slidingNum = 32

for s in tqdm_notebook( [Sensor1,Sensor2,Sensor3,Sensor4,Sensor5,Sensor6],leave=False):
    for axis in tqdm_notebook( ['AccX','AccY','AccZ'],leave=False ):
        FileName = CommonName+'-'+s['Name']+'-'+axis+\
        '-win='+str(windowNum).zfill(4)+'-sld='+str(slidingNum).zfill(4)
        Extraction(s,FileName,windowNum, slidingNum)
        time.sleep(1)
        
    FileNameTemp = CommonName+'-'+s['Name']+\
    '-win='+str(windowNum).zfill(4)+'-sld='+str(slidingNum).zfill(4)
    joblib.dump(Vector_SMA(FileNameTemp+'AccXYZ', s['AccZ'], s['AccY'], s['AccZ'], windowNum, slidingNum, WindowDataPath),
            FeaturePath+FileNameTemp+'_SMA.dump')    

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"
Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


this data had finished making
===========finished Making Vector of Mean===========
this data had finished making
===========finished Making Vector of Median===========
this data had finished making
===========finished Making Vector of Variance===========
this data had finished making
===========finished Making Vector of standard Deviation===========
this data had finished making
===========finished Making Vector of Max===========
this data had finished making
===========finished Making Vector of Max===========
this data had finished making
===========finished Making Vector of Range===========
this data had finished making
===========finished Making Vector of RMS===========
this data had finished making
===========finished Making Vector of FFT Real===========
this data had finished making
===========finished Making Vector of FFT Image===========
this data had finished making
===========finished Making Vector of Energy===========
this data had finished making
(24166, 126)
(24166, 126)
(3

KeyboardInterrupt: 

In [ ]:
%%time
windowNum = 256
slidingNum = 32

for s in tqdm_notebook( [Sensor1,Sensor2,Sensor3,Sensor4,Sensor5,Sensor6],leave=False):
    for saxis in tqdm_notebook( ['AccX','AccY','AccZ'],leave=False ):
        sFileName = CommonName+'-'+s['Name']+'-'+saxis+\
        '-win='+str(windowNum).zfill(4)+'-sld='+str(slidingNum).zfill(4)
        
        for d in tqdm_notebook( [Sensor1,Sensor2,Sensor3,Sensor4,Sensor5,Sensor6],leave=False ):
            for daxis in tqdm_notebook( ['AccX','AccY','AccZ'] ,leave=False):
                dFileName = CommonName+'-'+d['Name']+'-'+daxis+\
                '-win='+str(windowNum).zfill(4)+'-sld='+str(slidingNum).zfill(4)
                
                FileNameTemp = CommonName+'-'+s['Name']+'-'+saxis+'-'+d['Name']+'-'+daxis
                joblib.dump(Vector_Corr(sFileName, s[saxis],dFileName, d[daxis], 
                                windowNum, slidingNum, WindowDataPath),FeaturePath+FileNameTemp+'_Corr.dump')
                time.sleep(1)

In [26]:
def CalcurateFeature(windowNum,slidingNum):
    for s in tqdm_notebook( [Sensor1,Sensor2,Sensor3,Sensor4,Sensor5,Sensor6],leave=False):
        for axis in tqdm_notebook( ['AccX','AccY','AccZ'],leave=False ):
            FileName = CommonName+'-'+s['Name']+'-'+axis+\
            '-win='+str(windowNum).zfill(4)+'-sld='+str(slidingNum).zfill(4)
            Extraction(s,FileName,windowNum, slidingNum)
            time.sleep(1)
        
    FileNameTemp = CommonName+'-'+s['Name']+'-win='+str(windowNum).zfill(4)+'-sld='+str(slidingNum).zfill(4)
    joblib.dump(Vector_SMA(FileNameTemp+'AccXYZ', s['AccZ'], s['AccY'], s['AccZ'], windowNum, slidingNum, WindowDataPath),
            FeaturePath+FileNameTemp+'_SMA.dump')    

In [27]:
def CalcurateCorr(windowNum,slidingNumngNum):
    for s in tqdm_notebook( [Sensor1,Sensor2,Sensor3,Sensor4,Sensor5,Sensor6],leave=False):
        for saxis in tqdm_notebook( ['AccX','AccY','AccZ'],leave=False ):
            sFileName = CommonName+'-'+s['Name']+'-'+saxis+\
            +'-win='+str(windowNum).zfill(4)+'-sld='+str(slidingNum).zfill(4)
            
            for d in tqdm_notebook( [Sensor1,Sensor2,Sensor3,Sensor4,Sensor5,Sensor6],leave=False ):
                for daxis in tqdm_notebook( ['AccX','AccY','AccZ'] ,leave=False):
                    dFileName = CommonName+'-'+d['Name']+'-'+daxis\
                    +'-win='+str(windowNum).zfill(4)+'-sld='+str(slidingNum).zfill(4)
                    
                    FileNameTemp = CommonName+'-'+s['Name']+'-'+saxis+'-'+d['Name']+'-'+daxis\
                    +'-win='+str(windowNum).zfill(4)+'-sld='+str(slidingNum).zfill(4)
                    
                    joblib.dump(Vector_Corr(sFileName, s[saxis],dFileName, d[daxis], 
                                    windowNum, slidingNum, WindowDataPath),FeaturePath+FileNameTemp+'_Corr.dump')
                    time.sleep(1)

In [ ]:
CalcurateFeature(512,32)

In [ ]:
CalcurateFeature(512,512/8)

In [ ]:
CalcurateFeature(512,256)

In [ ]:
CalcurateFeature(1024,32)

In [ ]:
CalcurateFeature(1024,1024/8)

In [ ]:
CalcurateFeature(1024,512)

In [ ]:
CalcurateFeature(2048,32)

In [28]:
CalcurateFeature(2048,1024)

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"
Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


this data had finished making
===========finished Making Vector of Mean===========
this data had finished making
===========finished Making Vector of Median===========
this data had finished making
===========finished Making Vector of Variance===========
this data had finished making
===========finished Making Vector of standard Deviation===========
this data had finished making
===========finished Making Vector of Max===========
this data had finished making
===========finished Making Vector of Max===========
this data had finished making
===========finished Making Vector of Range===========
this data had finished making
===========finished Making Vector of RMS===========
this data had finished making
===========finished Making Vector of FFT Real===========
this data had finished making
===========finished Making Vector of FFT Image===========
this data had finished making


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/takeyama/.pyenv/versions/2.7.11/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/home/takeyama/.pyenv/versions/2.7.11/envs/takeyama/src/tqdm/tqdm/_tqdm.py", line 102, in run
    for instance in self.tqdm_cls._instances:
  File "/home/takeyama/.pyenv/versions/2.7.11/envs/takeyama/lib/python2.7/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



===========finished Making Vector of Energy===========
this data had finished making
(754, 1022)
(754, 1022)
(770588,)
===========finished Making Vector of Entropy===========
this data had finished making
(754, 2048)
(754, 2048)
(754, 1022)
===========finished Making Vector of Frequency===========
this data had finished making
===========finished Making Vector of Mean===========
this data had finished making
===========finished Making Vector of Median===========
this data had finished making
===========finished Making Vector of Variance===========
this data had finished making
===========finished Making Vector of standard Deviation===========
this data had finished making
===========finished Making Vector of Max===========
this data had finished making
===========finished Making Vector of Max===========
this data had finished making
===========finished Making Vector of Range===========
this data had finished making
===========finished Making Vector of RMS===========
this data had finis

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


...

===========finished Making Vector of Mean===========
this data had finished making
===========finished Making Vector of Median===========
this data had finished making
===========finished Making Vector of Variance===========
this data had finished making
===========finished Making Vector of standard Deviation===========
this data had finished making
===========finished Making Vector of Max===========
this data had finished making
===========finished Making Vector of Max===========
this data had finished making
===========finished Making Vector of Range===========
this data had finished making
===========finished Making Vector of RMS===========
this data had finished making
===========finished Making Vector of FFT Real===========
this data had finished making
===========finished Making Vector of FFT Image===========
this data had finished making
===========finished Making Vector of Energy===========
this data had finished making
(754, 1022)
(754, 1022)
(770588,)
===========finished Maki

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


===========finished Making Vector of Mean===========
this data had finished making
===========finished Making Vector of Median===========
this data had finished making
===========finished Making Vector of Variance===========
this data had finished making
===========finished Making Vector of standard Deviation===========
this data had finished making
===========finished Making Vector of Max===========
this data had finished making
===========finished Making Vector of Max===========
this data had finished making
===========finished Making Vector of Range===========
this data had finished making
===========finished Making Vector of RMS===========
this data had finished making
===========finished Making Vector of FFT Real===========
this data had finished making
===========finished Making Vector of FFT Image===========
this data had finished making
===========finished Making Vector of Energy===========
this data had finished making
(754, 1022)
(754, 1022)
(770588,)
===========finished Maki

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


===========finished Making Vector of Mean===========
this data had finished making
===========finished Making Vector of Median===========
this data had finished making
===========finished Making Vector of Variance===========
this data had finished making
===========finished Making Vector of standard Deviation===========
this data had finished making
===========finished Making Vector of Max===========
this data had finished making
===========finished Making Vector of Max===========
this data had finished making
===========finished Making Vector of Range===========
this data had finished making
===========finished Making Vector of RMS===========
this data had finished making
===========finished Making Vector of FFT Real===========
this data had finished making
===========finished Making Vector of FFT Image===========
this data had finished making
===========finished Making Vector of Energy===========
this data had finished making
(754, 1022)
(754, 1022)
(770588,)
===========finished Maki

In [29]:
CalcurateFeature(4096,32)

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"
Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


Participants07-Third-01-01_LeftHand-AccX-win=4096-sld=0032 is registed now
Build Complete
Source shape is(773550, 1)
window frames num =24046
offset=14
(24046, 4096, 1)
===========finished Making Vector of Mean===========
this data had finished making
===========finished Making Vector of Median===========
this data had finished making
===========finished Making Vector of Variance===========
this data had finished making
===========finished Making Vector of standard Deviation===========
this data had finished making
===========finished Making Vector of Max===========
this data had finished making
===========finished Making Vector of Max===========
this data had finished making
===========finished Making Vector of Range===========
Participants07-Third-01-01_LeftHand-AccX-win=4096-sld=0032 is registed now
Build Complete
Source shape is(773550, 1)
window frames num =24166
offset=14
(24166, 256, 1)
===========finished Making Vector of RMS===========
this data had finished making


KeyboardInterrupt: 

In [ ]:
CalcurateFeature(4096,2048)